In [1]:
from stacked_models import DeepLinear
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from time import perf_counter

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu'
activation = nn.ReLU()
epochs = 50
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [3]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
model = DeepLinear("SGC",{},in_feats, 16, n_classes, 2, 2, activation, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')

In [7]:
train(model, g, features, labels, train_mask, val_mask, test_mask)
test(model, g, features, labels, test_mask)

Epoch 01, Loss: 1.1829, Train: 0.5667, Val: 0.4920, Test: 0.5060
Epoch 02, Loss: 1.5257, Train: 0.6333, Val: 0.5200, Test: 0.5160
Epoch 03, Loss: 1.1014, Train: 0.6167, Val: 0.5080, Test: 0.5060
Epoch 04, Loss: 0.8675, Train: 0.7833, Val: 0.6600, Test: 0.6550
Epoch 05, Loss: 0.7481, Train: 0.8500, Val: 0.6280, Test: 0.6180
Epoch 06, Loss: 0.7219, Train: 0.8500, Val: 0.6260, Test: 0.6030
Epoch 07, Loss: 0.6611, Train: 0.9000, Val: 0.6480, Test: 0.6200
Epoch 08, Loss: 0.5960, Train: 0.9000, Val: 0.6840, Test: 0.6570
Epoch 09, Loss: 0.5501, Train: 0.8833, Val: 0.7000, Test: 0.6740
Epoch 10, Loss: 0.4971, Train: 0.9000, Val: 0.7140, Test: 0.6870
Epoch 11, Loss: 0.4545, Train: 0.9167, Val: 0.7200, Test: 0.7060
Epoch 12, Loss: 0.4322, Train: 0.9167, Val: 0.7280, Test: 0.7170
Epoch 13, Loss: 0.4235, Train: 0.9167, Val: 0.7260, Test: 0.7160
Epoch 14, Loss: 0.4279, Train: 0.9167, Val: 0.7240, Test: 0.7070
Epoch 15, Loss: 0.3448, Train: 0.9167, Val: 0.7200, Test: 0.7040
Epoch 16, Loss: 0.2872, T

0.765